In [1]:
import os
import time
import math

from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.optimizers import adam
from keras.utils import np_utils
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from keras import backend as K

from utils import load_data_to_mem, augmentation

K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
train_path = os.path.join(".", "data", "train")
valid_path = os.path.join(".", "data", "val")
test_path = os.path.join(".", "data", "test")

classes = ["I", "II", "III", "IV", "V", "VI", "VII", "VIII"]
class_mapping = {"I": 0, "II": 1, "III": 2, "IV": 3,
                 "V": 4, "VI": 5, "VII": 6, "VIII":7}
img_rows, img_cols = 64, 64

def class_to_number(n):
    return class_mapping[n]

X_train, y_train = load_data_to_mem(train_path, classes, 
                                    img_height=img_rows, img_width=img_cols)
X_valid, y_valid = load_data_to_mem(valid_path, classes,
                                    img_height=img_rows, img_width=img_cols)
X_test, y_test = load_data_to_mem(test_path, classes,
                                  img_height=img_rows, img_width=img_cols)

X_train, y_train = augmentation(X_train, y_train, n_transform=30)
# test more data
#X_train, y_train = augmentation(X_train + X_test, y_train + y_test)

In [3]:
X_train = np.array(X_train, dtype=np.float32) / 255
X_valid = np.array(X_valid, dtype=np.float32) / 255
X_test = np.array(X_test, dtype=np.float32) / 255
y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

# expand dimensions for CNN
X_train = np.expand_dims(X_train, axis=3)
X_valid = np.expand_dims(X_valid, axis=3)
X_test = np.expand_dims(X_test, axis=3)

# expand dimensions for MultiLabelBinarizer
y_train = np.expand_dims(y_train, axis=1)
y_valid = np.expand_dims(y_valid, axis=1)
y_test = np.expand_dims(y_test, axis=1)

mlb = MultiLabelBinarizer()
mlb.fit(y_train)
Y_train = mlb.transform(y_train)
Y_valid = mlb.transform(y_valid)
Y_test = mlb.transform(y_test)


In [4]:
print("Check shapes: ")
print("X_train =", X_train.shape, " Y_train =", Y_train.shape)
print("X_valid =", X_valid.shape, " Y_valid =", Y_valid.shape)
print("X_test =", X_test.shape, " Y_test =", Y_test.shape)


Check shapes: 
X_train = (30690, 64, 64, 1)  Y_train = (30690, 8)
X_valid = (328, 64, 64, 1)  Y_valid = (328, 8)
X_test = (338, 64, 64, 1)  Y_test = (338, 8)


In [9]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation="relu",
                 input_shape=(img_rows, img_cols, 1)))
# model.add(Conv2D(32, kernel_size=(5, 5), activation="relu"))
#model.add(Conv2D(32, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=(3, 3), activation="relu"))
#model.add(Conv2D(64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=(3, 3), activation="relu"))
#model.add(Conv2D(128, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
#model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(8, activation="softmax"))

model.compile(loss="categorical_crossentropy", 
              optimizer="adam",
             metrics=["accuracy"])

# callbacks = [TensorBoard(log_dir="tb_logs", histogram_freq=1, write_images=True)]

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 12, 12, 128)       73856     
__________

In [14]:
start_time = time.time()
history = model.fit(X_train, Y_train,
                   batch_size=64,
                   epochs=5,
                   validation_data=(X_valid, Y_valid),
                   verbose=2)

train_time = time.time() - start_time
print("Total training time =", train_time)


Train on 30690 samples, validate on 328 samples
Epoch 1/5
 - 67s - loss: 0.6971 - acc: 0.7545 - val_loss: 0.6464 - val_acc: 0.7835
Epoch 2/5
 - 72s - loss: 0.6961 - acc: 0.7588 - val_loss: 0.6369 - val_acc: 0.7683
Epoch 3/5
 - 67s - loss: 0.6718 - acc: 0.7682 - val_loss: 0.6573 - val_acc: 0.7866
Epoch 4/5
 - 64s - loss: 0.6645 - acc: 0.7703 - val_loss: 0.6180 - val_acc: 0.7988
Epoch 5/5
 - 65s - loss: 0.6488 - acc: 0.7749 - val_loss: 0.6424 - val_acc: 0.7896
Total training time = 335.2411665916443


In [15]:
scores = model.evaluate(X_test, Y_test)
print(scores)
print("Test data accuracy =", scores[1])


338/338 [==============================] - 0s 1ms/step
[0.6733211345926544, 0.7603550306438694]
Test data accuracy = 0.7603550306438694


In [12]:
model.save_weights("logs/model3_weights.hdf5")
saved_model = model.to_json()
with open("logs/model3_architecture.json", "w") as json_file:
    json_file.write(saved_model)